# Building Q-Table for Choosing Cards

In [5]:
from tqdm import tqdm
from card_choice_gym import CardChoiceEnv
from utils import *
import numpy as np
import random
from keras.models import load_model
import time
import os

In [2]:
q = np.load('../models/q-table.npy')

In [2]:
if os.path.exists('../models/calling_nn_1.pickle'):
    cm = load_model('../models/calling_nn_1.pickle')


2022-05-15 15:45:11.613607: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def LearnJoker(q_in=np.zeros((4, 9, 3, 4, 19, 4)), alpha_in=0.01, epsilon_in=0.5, gamma_in=0.95, episodes_in=100):
  acts = ['STRG-BEAT', 'STRG-LOSS', 'WEAK-BEAT','WEAK-LOSS']
  env = CardChoiceEnv()

  alpha, gamma, epsilon = alpha_in, gamma_in, epsilon_in
  q = q_in
  
  wins = []
  good_calls = np.zeros((7,))

  actions = []

  for i in tqdm(range(episodes_in)): 
    done = False
    s = env.reset()
    env.calling_model = cm
    s0, s1, s2, s3, s4 = s
    while True: 
      if np.random.random() < epsilon:
      # choose random action
        act_num = random.randint(0, 3)
      else:
        # greedy
        act_num = np.argmax(q[s0, s1, s2, s3, s4])
      
      action = acts[act_num]

      s_, r, done, _ = env.step(action)

      s_0, s_1, s_2, s_3, s_4 = s_
      
      td_target = r + gamma * np.argmax(q[s_0, s_1, s_2, s_3, s_4])
      td_error = td_target - q[s0, s1, s2, s3, s4, act_num]
      s = s_

      q[s0, s1, s2, s3, s4, act_num] += alpha * td_error
      if done:
        if r > 0:
          wins.append(i)
          good_calls = np.vstack((good_calls, [env.call_state]))
        break
      actions.append(act_num)
  return wins, good_calls, q, actions

In [7]:

start = time.perf_counter()
eps = 1000
wins, calls, q, acts= LearnJoker(epsilon_in=1, episodes_in=eps)
print(f"Win rate: {len(wins)/ eps}")
end = time.perf_counter()
print(f"Time: {end - start} seconds")


100%|██████████| 1000/1000 [00:11<00:00, 88.19it/s]

Win rate: 0.102
Time: 11.366655232 seconds


In [25]:
start = time.perf_counter()
eps = 1000
wins, calls, q, actions = LearnJoker(epsilon_in=0, episodes_in=eps, q_in=q)
print(f"Win rate: {len(wins)/eps}")
end = time.perf_counter()
print(f"Time: {end - start} seconds")


Win rate: 0.319
Time: 2.9014058640000258 seconds


In [33]:
from more_itertools import quantify
[quantify([i == j for i in actions]) for j in range(4)]

[75, 8, 3501, 4416]

In [34]:
save_q_table(q)
write_calls(calls)

NameError: name 'save_q_table' is not defined